In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import os

In [3]:
DATA_PATH = os.path.join('..', 'data')
PRIMARY_PATH = os.path.join(DATA_PATH, 'primary')

In [4]:
df = pd.read_parquet(os.path.join(PRIMARY_PATH, 'pres-1990-2014.parquet'))
df

,eleccion,tipo,region,electoral,candidato,votos,porcentaje
0,1990,0,tarapaca,ARICA,Francisco J. Errázuriz,14981,0.175580
1,1990,0,tarapaca,ARICA,Hernán Büchi,25786,0.302216
2,1990,0,tarapaca,ARICA,Patricio Aylwin,44556,0.522204
3,1990,0,tarapaca,BELEN,Francisco J. Errázuriz,81,0.270903
4,1990,0,tarapaca,BELEN,Hernán Büchi,143,0.478261
...,...,...,...,...,...,...,...
21402,2014,2,magallanes,SAN GREGORIO,Michelle Bachelet,108,0.624277
21403,2014,2,magallanes,TIMAUKEL,Evelyn Matthei,42,0.506024
21404,2014,2,magallanes,TIMAUKEL,Michelle Bachelet,41,0.493976
21405,2014,2,magallanes,TORRES DEL PAINE (C. CASTILLO),Evelyn Matthei,147,0.389920


# Bellwether Electorales

### Bellwether del Ganador

In [75]:
primera = df.query('tipo == 1').drop(columns=['tipo'])
primera_bw = primera.loc[primera.groupby(['eleccion', 'region', 'electoral']).votos.idxmax()]
segunda = df.query('tipo == 2').drop(columns=['tipo'])
segunda_bw = segunda.loc[segunda.groupby(['eleccion', 'region', 'electoral']).votos.idxmax()]

In [76]:
bw_ganador = primera_bw.merge(segunda_bw, on=['eleccion', 'region', 'electoral'], suffixes=('_1', '_2'))
winners = {
    2000: 'Ricardo Lagos',
    2006: 'Michelle Bachelet',
    2010: 'Sebastián Piñera',
    2014: 'Michelle Bachelet',
}
bw_ganador['ganador'] = bw_ganador['eleccion'].map(winners)
bw_ganador['bw'] = bw_ganador.candidato_1 == bw_ganador.ganador
bw_ganador

,eleccion,region,electoral,candidato_1,votos_1,porcentaje_1,candidato_2,votos_2,porcentaje_2,ganador,bw
0,2000,aisen,AISEN,Ricardo Lagos,4032,0.532840,Ricardo Lagos,4254,0.553834,Ricardo Lagos,True
1,2000,aisen,BAHIA MURTA,Joaquín Lavín,209,0.794677,Joaquín Lavín,216,0.797048,Ricardo Lagos,False
2,2000,aisen,BALMACEDA,Joaquín Lavín,213,0.577236,Joaquín Lavín,216,0.583784,Ricardo Lagos,False
3,2000,aisen,CHILE CHICO,Ricardo Lagos,1002,0.543384,Ricardo Lagos,1100,0.588235,Ricardo Lagos,True
4,2000,aisen,CISNES,Ricardo Lagos,491,0.520679,Ricardo Lagos,526,0.537283,Ricardo Lagos,True
...,...,...,...,...,...,...,...,...,...,...,...
2192,2014,valparaiso,SANTA MARIA,Michelle Bachelet,3689,0.589297,Michelle Bachelet,4112,0.732063,Michelle Bachelet,True
2193,2014,valparaiso,SANTO DOMINGO,Michelle Bachelet,2697,0.405869,Evelyn Matthei,2988,0.502523,Michelle Bachelet,True
2194,2014,valparaiso,SAUSALITO,Evelyn Matthei,18248,0.418868,Evelyn Matthei,22741,0.625406,Michelle Bachelet,False
2195,2014,valparaiso,VILLA ALEMANA,Michelle Bachelet,14989,0.364847,Michelle Bachelet,18078,0.534583,Michelle Bachelet,True


In [77]:
bw = bw_ganador.groupby(['region', 'electoral']).bw.sum().reset_index().query('bw == 4')
bw.shape

(119, 3)

119 de las 604 Circunscripciones Electorales fueron Bellwether del Ganador en todas las elecciones presidenciales entre el 2000 y el 2014 (4 elecciones).

In [81]:
bw[['region', 'electoral']].sort_values(['region', 'electoral'])

,region,electoral
0,aisen,AISEN
3,aisen,CHILE CHICO
4,aisen,CISNES
15,aisen,PUERTO CHACABUCO
17,aisen,PUYUHUAPI
...,...,...
584,valparaiso,PANQUEHUE
594,valparaiso,RINCONADA
595,valparaiso,SAN ANTONIO
597,valparaiso,SAN FELIPE


In [88]:
primera_bw_counts = primera[primera.region.isin(bw.region) & primera.electoral.isin(bw.electoral)].copy()
primera_bw_counts.sort_values(['region', 'electoral'], inplace=True)
primera_bw_counts.groupby('eleccion').votos.sum()/primera.groupby('eleccion').votos.sum()

eleccion
2000    0.450189
2006    0.441040
2010    0.432499
2014    0.429587
Name: votos, dtype: float64

In [89]:
segunda_bw_counts = segunda[segunda.region.isin(bw.region) & segunda.electoral.isin(bw.electoral)].copy()
segunda_bw_counts.sort_values(['region', 'electoral'], inplace=True)
segunda_bw_counts.groupby('eleccion').votos.sum()/segunda.groupby('eleccion').votos.sum()

eleccion
2000    0.448389
2006    0.439063
2010    0.429766
2014    0.420697
Name: votos, dtype: float64

De las 119 Circuncripciones Electorales "Bellwether del Ganador", representan entre el 42-45% de los votos totales en las elecciones presidenciales.

# Swing Electorales

In [56]:
izq_der = {
    'Francisco J. Errázuriz': 'derecha',
    'Hernán Büchi': 'derecha',
    'Patricio Aylwin': 'izquierda',
    'Cristian Reitze': 'izquierda',
    'Eugenio Pizarro': 'izquierda',
    'Manfred Max Neef': 'izquierda',
    'José Piñera': 'derecha',
    'Arturo Alessandri': 'derecha',
    'Eduardo Frei': 'izquierda',
    'Arturo Frei': 'izquierda',
    'Sara Larráin': 'izquierda',
    'Gladys Marín': 'izquierda',
    'Tomás Hirsch': 'izquierda',
    'Ricardo Lagos': 'izquierda',
    'Joaquín Lavín': 'derecha',
    'Sebastián Piñera': 'derecha',
    'Michelle Bachelet': 'izquierda',
    'Jorge Arrate': 'izquierda',
    'Marco Enríquez - Ominami ': 'izquierda',
    'Tomás Jocelyn - Holt': 'izquierda',
    'Ricardo Israel': 'derecha',
    'Roxana Miranda ': 'izquierda',
    'Alfredo Sfeir ': 'izquierda',
    'Marcel Claude': 'izquierda',
    'Franco Parisi ': 'derecha',
    'Evelyn Matthei': 'derecha'
}

# -5 izquierda / 0 centro / 5 derecha
ideologia = {
    'Francisco J. Errázuriz': 4,
    'Hernán Büchi': 5,
    'Patricio Aylwin': -1,
    'Cristian Reitze': -2,
    'Eugenio Pizarro': -4,
    'Manfred Max Neef': -3,
    'José Piñera': 5,
    'Arturo Alessandri': 3,
    'Eduardo Frei': -1,
    'Arturo Frei': -2,
    'Sara Larráin': -3,
    'Gladys Marín': -5,
    'Tomás Hirsch': -4,
    'Ricardo Lagos': -1,
    'Joaquín Lavín': 3,
    'Sebastián Piñera': 2,
    'Michelle Bachelet': -2,
    'Jorge Arrate': -4,
    'Marco Enríquez - Ominami ': -1,
    'Tomás Jocelyn - Holt': 0,
    'Ricardo Israel': 0,
    'Roxana Miranda ': -5,
    'Alfredo Sfeir ': -2,
    'Marcel Claude': -4,
    'Franco Parisi ': 1,
    'Evelyn Matthei': 4
}

swing = bw_ganador.drop(columns=['bw'])
swing['izq_der_1'] = swing['candidato_1'].map(izq_der)
swing['izq_der_2'] = swing['candidato_2'].map(izq_der)
swing['ideologia_1'] = swing['candidato_1'].map(ideologia)
swing['ideologia_2'] = swing['candidato_2'].map(ideologia)
swing

,eleccion,region,electoral,candidato_1,votos_1,porcentaje_1,candidato_2,votos_2,porcentaje_2,ganador,izq_der_1,izq_der_2,ideologia_1,ideologia_2
0,2000,aisen,AISEN,Ricardo Lagos,4032,0.532840,Ricardo Lagos,4254,0.553834,Ricardo Lagos,izquierda,izquierda,-1,-1
1,2000,aisen,BAHIA MURTA,Joaquín Lavín,209,0.794677,Joaquín Lavín,216,0.797048,Ricardo Lagos,derecha,derecha,3,3
2,2000,aisen,BALMACEDA,Joaquín Lavín,213,0.577236,Joaquín Lavín,216,0.583784,Ricardo Lagos,derecha,derecha,3,3
3,2000,aisen,CHILE CHICO,Ricardo Lagos,1002,0.543384,Ricardo Lagos,1100,0.588235,Ricardo Lagos,izquierda,izquierda,-1,-1
4,2000,aisen,CISNES,Ricardo Lagos,491,0.520679,Ricardo Lagos,526,0.537283,Ricardo Lagos,izquierda,izquierda,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,2014,valparaiso,SANTA MARIA,Michelle Bachelet,3689,0.589297,Michelle Bachelet,4112,0.732063,Michelle Bachelet,izquierda,izquierda,-2,-2
2193,2014,valparaiso,SANTO DOMINGO,Michelle Bachelet,2697,0.405869,Evelyn Matthei,2988,0.502523,Michelle Bachelet,izquierda,derecha,-2,4
2194,2014,valparaiso,SAUSALITO,Evelyn Matthei,18248,0.418868,Evelyn Matthei,22741,0.625406,Michelle Bachelet,derecha,derecha,4,4
2195,2014,valparaiso,VILLA ALEMANA,Michelle Bachelet,14989,0.364847,Michelle Bachelet,18078,0.534583,Michelle Bachelet,izquierda,izquierda,-2,-2


In [91]:
safe = swing.value_counts(['region', 'electoral', 'izq_der_1']).reset_index().query('count >= 4')
safe.shape[0]

108

In [102]:
safe_counts = swing[swing.region.isin(safe.region) & swing.electoral.isin(safe.electoral)].copy()
safe_counts.groupby('eleccion').sum()[['votos_1', 'votos_2']]/swing.groupby('eleccion').sum()[['votos_1', 'votos_2']]

,votos_1,votos_2
eleccion,,
2000,0.150858,0.150058
2006,0.141117,0.150262
2010,0.149826,0.154926
2014,0.163378,0.159784


108 de las 604 Circunscripciones Electorales han votado en primera vuelta por un candidato de la misma ideología política en todas las elecciones presidenciales entre el 2000 y el 2014 (4 elecciones).

Estas 108 Circunscripciones Electorales Seguras representan entre el 14-16% de los votos totales en las elecciones presidenciales.

In [103]:
casi_safe = swing.value_counts(['region', 'electoral', 'izq_der_1']).reset_index().query('count >= 3')
casi_safe.shape[0]

319

In [104]:
casi_safe_counts = swing[swing.region.isin(casi_safe.region) & swing.electoral.isin(casi_safe.electoral)].copy()
casi_safe_counts.groupby('eleccion').sum()[['votos_1', 'votos_2']]/swing.groupby('eleccion').sum()[['votos_1', 'votos_2']]

,votos_1,votos_2
eleccion,,
2000,0.654567,0.660452
2006,0.675829,0.670018
2010,0.621580,0.643042
2014,0.655681,0.658672


Pero, 319 de las 604 Circunscripciones Electorales han votado al menos 3 veces por un candidato de la misma ideología política en las elecciones presidenciales entre el 2000 y el 2014 (4 elecciones).

Estas 319 Circunscripciones Electorales Casi Seguras representan entre el 62-68% de los votos totales en las elecciones presidenciales.